In [1]:
import os,random
import csv
os.environ["KERAS_BACKEND"] = "tensorflow"
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import numpy as np
from copy import deepcopy
from tensorflow.keras.models import load_model
import tensorflow.keras.models as models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, AlphaDropout, GlobalAveragePooling1D, BatchNormalization, add,LSTM,GRU,Bidirectional, Attention
from tensorflow.python.keras.layers.core import Flatten, Dense, Dropout, Reshape, Activation,Lambda,Permute
from tensorflow.python.keras.layers.convolutional import Conv1D,Conv2D, AveragePooling1D, ZeroPadding2D, Convolution2D, MaxPooling1D, MaxPooling2D
from tensorflow.keras.regularizers import *
from tensorflow.keras import layers
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import pickle,time
import random, sys
from tensorflow import keras
from tensorflow.keras import backend as K
import tensorflow as tf
mirrored_strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [2]:
Xd = pickle.load(open('/media/ext_storage/ahmed_backup/ML/RML22-main/code/RML22_org.pickle', 'rb'))
snrs,mods = map(lambda j: sorted( list( set( map( lambda x: x[j], Xd.keys() ) ) ) ), [1,0])

X = []
Y_snr = []
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
    Y_snr = Y_snr + [mod]*120000
X = np.vstack(X)
clip_value = max(abs(float(np.min(X))), float(np.max(X)))
print(X.shape)
Y_snr = np.vstack(Y_snr)

(1260000, 2, 128)


In [3]:
def to_onehot(yin):
    yy = list(yin)
    yy1 = np.zeros([len(yy), max(yy)+1])
    yy1[np.arange(len(yy)),yy] = 1
    return yy1
Y_snr = to_onehot(map(lambda x: mods.index(lbl[x][0]), range(X.shape[0])))

In [4]:
# Partition the dataset into training and testing datasets
np.random.seed(2016)
n_examples = X.shape[0]
n_train    = int(round(n_examples * 0.5))
train_idx  = np.random.choice(range(0,n_examples), size=n_train, replace=False)
test_idx   = list(set(range(0,n_examples))-set(train_idx))
X_train    = X[train_idx]
X_test     = X[test_idx]

Y_train = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), train_idx)))
Y_test = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))

in_shp = list(X_train.shape[1:])
output_shp = Y_train.shape[1]
print(f'Network input shape in variable input-shp: {in_shp}')
print(f'Network output shape in variable input-shp: {output_shp}')
print(X_train.shape, in_shp)
classes = mods

Network input shape in variable input-shp: [2, 128]
Network output shape in variable input-shp: 10
(630000, 2, 128) [2, 128]


In [5]:
X_train_orig = deepcopy(X_train)
X_test_orig = deepcopy(X_test)

num_samples_list = [16, 32, 64]
field_names= ['num_subsample'] + list(snrs)

In [ ]:
ranker_types = ["complex_cnn", "resnet", "cldnn"]

for ranker_type in ranker_types:
    if ranker_type == "resnet":
        # build model ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        def residual_stack(x):
            def residual_unit1(y,_strides=1):
                shortcut_unit=y
                # 1x1 conv linear
                y = layers.Conv1D(8, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
                y = layers.BatchNormalization()(y)

                y = layers.Conv1D(8, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
                y = layers.BatchNormalization()(y)
                # add batch normalization
                y = layers.add([shortcut_unit,y])
                return y
    
            def residual_unit2(y,_strides=1):
                shortcut_unit=y
                # 1x1 conv linear
                y = layers.Conv1D(16, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
                y = layers.BatchNormalization()(y)
                y = layers.Conv1D(16, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                          kernel_initializer='glorot_uniform')(y)
                y = layers.BatchNormalization()(y)
                # add batch normalization
                y = layers.add([shortcut_unit,y])
                return y
    
            def residual_unit3(y,_strides=1):
                shortcut_unit=y
                # 1x1 conv linear
                y = layers.Conv1D(32, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
                y = layers.BatchNormalization()(y)
                y = layers.Conv1D(32, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
                y = layers.BatchNormalization()(y)
                # add batch normalization
                y = layers.add([shortcut_unit,y])
                return y
    
            def residual_unit4(y,_strides=1):
                shortcut_unit=y
                # 1x1 conv linear
                y = layers.Conv1D(64, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
                y = layers.BatchNormalization()(y)
                y = layers.Conv1D(64, kernel_size=7,data_format='channels_first',strides=_strides,padding='same',activation='relu',
                         kernel_initializer='glorot_uniform')(y)
                y = layers.BatchNormalization()(y)
                # add batch normalization
                y = layers.add([shortcut_unit,y])
                return y
    
            x = layers.Conv1D(8, data_format='channels_first',kernel_size=1, padding='same',activation='relu',
                      kernel_initializer='glorot_uniform')(x)
            x = layers.BatchNormalization()(x)

            x = residual_unit1(x)
            x = layers.Conv1D(16, data_format='channels_first',kernel_size=1, padding='same',activation='relu',
                     kernel_initializer='glorot_uniform')(x)
            x = layers.BatchNormalization()(x)
            x = residual_unit2(x)
            x = layers.Conv1D(32, data_format='channels_first',kernel_size=1, padding='same',activation='relu',
                     kernel_initializer='glorot_uniform')(x)
            x = layers.BatchNormalization()(x)
            x = residual_unit3(x)
            x = layers.Conv1D(64, data_format='channels_first',kernel_size=1, padding='same',activation='relu',
                     kernel_initializer='glorot_uniform')(x)
            x = layers.BatchNormalization()(x)
            x = residual_unit4(x)
            x = layers.AveragePooling1D(data_format='channels_first',pool_size=in_shp[1])(x)
            return x
    
        inputs=layers.Input(shape=in_shp)
        x = residual_stack(inputs)
        x = Flatten()(x)
        x = Dense(256,kernel_initializer="he_normal", activation="selu", name="dense1")(x)
        x = AlphaDropout(0.5)(x)
        x = Dense(len(classes),kernel_initializer="he_normal", activation="softmax", name="dense3")(x)
        model = models.Model(inputs,x)
        model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(lr=0.001),metrics=['accuracy'])
        model.summary()


    elif ranker_type == "cldnn":
        dr = 0.5 # dropout rate (%)
        model = models.Sequential()
        model.add(Reshape([1] + input_shp, input_shape=input_shp))
        model.add(ZeroPadding2D((0, 2), data_format="channels_first"))
        model.add(Conv2D(kernel_initializer="glorot_uniform", name="conv1", activation="relu", data_format="channels_first", padding="valid", filters=256, kernel_size=(1, 3)))
        model.add(Dropout(dr))

        model.add(ZeroPadding2D((0, 2), data_format="channels_first"))
        model.add(Conv2D(kernel_initializer="glorot_uniform", name="conv2", activation="relu", data_format="channels_first", padding="valid", filters=256, kernel_size=(2, 3)))
        model.add(Dropout(dr))

        model.add(ZeroPadding2D((0, 2), data_format="channels_first"))
        model.add(Conv2D(kernel_initializer="glorot_uniform", name="conv3", activation="relu", data_format="channels_first", padding="valid", filters=80, kernel_size=(1, 3)))
        model.add(Dropout(dr))

        model.add(ZeroPadding2D((0, 2), data_format="channels_first"))
        model.add(Conv2D(kernel_initializer="glorot_uniform", name="conv4", activation="relu", data_format="channels_first", padding="valid", filters=80, kernel_size=(1, 3)))
        model.add(Dropout(dr))
        model.add(ZeroPadding2D((0, 2), data_format="channels_first"))

        model.add(Flatten())

        model.add(Reshape((1,11200)))
        model.add(LSTM(50))
        model.add(Dense(256, activation='relu', kernel_initializer='he_normal', name="dense1"))
        model.add(Dropout(dr))
        model.add(Dense( len(classes), kernel_initializer='he_normal', name="dense2" ))
        model.add(Activation('softmax'))
        model.add(Reshape([len(classes)]))
        model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(lr=0.0001),metrics=['accuracy'])
        model.summary()
        
    else:        
        dr=0.5
        model = Sequential()
        model.add(Reshape(input_shp+[1], input_shape=input_shp))
        model.add(ZeroPadding2D((1, 2)))
        model.add(Convolution2D(256, (2, 3), padding='valid', activation='relu', name="conv1", kernel_initializer='glorot_uniform'))#ch from 3->4
        model.add(BatchNormalization())
        model.add(Permute((3,2,1)))
        model.add(Lambda(LC))
        model.add(Permute((3,2,1)))
        model.add(Activation('relu'))
        model.add(Dropout(dr))
        model.add(Convolution2D(80, (2, 3), padding='valid', activation="relu", name="conv2", kernel_initializer='glorot_uniform'))
        model.add(BatchNormalization())
        model.add(Dropout(dr))
        model.add(Flatten())
        model.add(Dense(256, activation='relu', kernel_initializer='he_normal', name="dense1"))
        model.add(Dropout(dr))
        model.add(Dense( len(classes), kernel_initializer='he_normal', name="dense2" ))
        model.add(Activation('softmax'))
        model.add(Reshape([len(classes)]))
        model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(lr=0.0001),metrics=['accuracy'])
        model.summary()
        
        
    # Number of epochs
    epochs = 200
    # Training batch size
    batch_size = 512
    
    #train Complex
    start = time.time()
    #filepath = 'complex.wts.h5'
    filepath = 'ranker_' + ranker_type + '.wts.h5'

    history = model.fit(X_train,Y_train,batch_size=batch_size,epochs=epochs,verbose=2,validation_split = 0.2,
        callbacks = [
            tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto'),
            tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=15, verbose=1, mode='auto')
        ])
    end = time.time()
    duration = end - start
    print('Training time = ' + str(round(duration/60,5)) + 'minutes')
    
    
    field_names= ['num_subsample'] + list(snrs)
    acc = {}
    acc["num_subsample"] = 128
    for snr in snrs:
        # extract classes @ SNR
        print(snr)
        test_SNRs = list(map(lambda x: lbl[x][1], test_idx))
        test_X_i = X_test[np.where(np.array(test_SNRs)==snr)]
        test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]

        test_Y_i_hat = model.predict(test_X_i, batch_size=batch_size)
        conf = np.zeros([len(classes),len(classes)])
        confnorm = np.zeros([len(classes),len(classes)])
        for i in range(0,test_X_i.shape[0]):
            j = list(test_Y_i[i,:]).index(1)
            k = int(np.argmax(test_Y_i_hat[i,:]))
            conf[j,k] = conf[j,k] + 1
        for i in range(0,len(classes)):
            confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])  
        cor = np.sum(np.diag(conf))
        ncor = np.sum(conf) - cor
        print ("Overall Accuracy: ", cor / (cor+ncor))
        acc[snr] = 1.0*cor/(cor+ncor)

    num_samples = 128
    # Save results to a pickle file for plotting later
    print (acc)
    with open('./results/acc_' + ranker_type + '_Ranker_' + str(num_samples) + '.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=field_names)
        writer.writeheader()
        writer.writerow(acc)
